# AI cập nhật dữ liệu mới nhất cho báo cáo tuần

In [36]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import get_and_crawl_data
import gemini_model
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(get_and_crawl_data)
importlib.reload(gemini_model)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from get_and_crawl_data import *
from gemini_model import *
from plotly_and_upload import *

- Lấy dữ liệu từ Mongo

In [5]:
date_series = get_mongo_collection(ref_db, 'date_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

In [6]:
projection = {
    "_id": 0,
    "date": 1,
    "ticker": 1,
    "open": 1,
    "high": 1,
    "low": 1,
    "close": 1,
    "volume": 1,
    "option": 1,
    "SMA_20": 1,
    "SMA_60": 1,
    "RSI_14": 1,
    "week_prev_high": 1,
    "week_prev_low": 1,
    "week_open": 1,
    "month_prev_high": 1,
    "month_prev_low": 1,
    "month_open": 1,
    "quarter_prev_high": 1,
    "quarter_prev_low": 1,
    "quarter_open": 1,
    "WFIBO_0382": 1,
    "WFIBO_0500": 1,
    "WFIBO_0618": 1,
    "MFIBO_0382": 1,
    "MFIBO_0500": 1,
    "MFIBO_0618": 1,
    "QFIBO_0382": 1,
    "QFIBO_0500": 1,
    "QFIBO_0618": 1,
}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

In [7]:
projection = {"_id": 0,"date": 1,"ticker": 1,"trend_5p": 1,"trend_20p": 1,"trend_60p": 1,"trend_240p": 1}
today_ms_chart_df = get_mongo_collection(stock_db, "today_ms_chart", find_query={"ticker":'all'}, projection=projection)
history_ms_chart_df = get_mongo_collection(stock_db, "history_ms_chart", find_query={"ticker":'all'}, projection=projection)
full_ms_chart_df = pd.concat([today_ms_chart_df, history_ms_chart_df], axis=0, ignore_index=True).drop(columns=['ticker'])

projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1,"volume": 1,'cap': 1,'t5_score':1}
today_stock_df = get_mongo_collection(stock_db, "today_stock", find_query={"date": {"$in": date_series['date'].iloc[:5].tolist()}}, projection=projection)
history_stock_df = get_mongo_collection(stock_db, "history_stock", find_query={"date": {"$in": date_series['date'].iloc[:5].tolist()}}, projection=projection)
full_stock_df = pd.concat([today_stock_df, history_stock_df], axis=0, ignore_index=True)

other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

- Lấy dữ liệu từ AlphaVantage

In [8]:
# Lấy dữ liệu các tỷ giá cần thiết từ Alpha Vantage
fx_pairs = [
    ('USD', 'VND', 'USD_VND'),
    ('USD', 'SEK', 'USD_SEK')
]

data_frames = []
for from_curr, to_curr, col_name in fx_pairs:
    df = get_data_from_av(from_curr, to_curr, col_name)
    if df is not None:
        data_frames.append(df)

av_ticker_df = pd.concat(data_frames, axis=1)
av_ticker_df = av_ticker_df.fillna(method='ffill').dropna()

### Phần bối cảnh vĩ mô

#### Dữ liệu các biểu đồ biến động

- Tính toán DXY từ các cặp tiền

In [9]:
currency_config = {
    'EUR_USD': {'source': 'other_ticker_df', 'weight': -0.576},
    'USD_JPY': {'source': 'other_ticker_df', 'weight': 0.136},
    'GBP_USD': {'source': 'other_ticker_df', 'weight': -0.119},
    'USD_CAD': {'source': 'other_ticker_df', 'weight': 0.091},
    'USD_CHF': {'source': 'other_ticker_df', 'weight': 0.036},
    'USD_SEK': {'source': 'av_ticker_df', 'weight': 0.042}
}

# Tạo DataFrame cho mỗi cặp tiền tệ
currency_data = {}
for pair, config in currency_config.items():
    if config['source'] == 'other_ticker_df':
        df = other_ticker_df[other_ticker_df['ticker'] == pair][['date', 'close']].set_index('date')
        df.columns = [pair]
    else:
        df = av_ticker_df[['USD_SEK']].copy()
        df.columns = [pair]
    
    currency_data[pair] = df

# Kết hợp tất cả dữ liệu tỷ giá
dxy_calculation_df = pd.concat(currency_data.values(), axis=1, sort=True)
dxy_calculation_df = dxy_calculation_df.sort_index().bfill().ffill()

# Tính toán chỉ số DXY theo công thức chuẩn
dxy_base = 50.14348112
dxy_calculation = dxy_base
for pair, config in currency_config.items():
    dxy_calculation *= (dxy_calculation_df[pair] ** config['weight'])

dxy_calculation_df['ticker'] = 'DXY'
dxy_calculation_df['close'] = dxy_calculation
dxy_calculation_df = dxy_calculation_df[['ticker', 'close']].reset_index()

- Chuẩn bị tỉ giá USD/VND

In [10]:
usd_vnd_df = av_ticker_df[['USD_VND']].rename(columns={'USD_VND': 'close'})
usd_vnd_df['ticker'] = 'USD_VND'
usd_vnd_df = usd_vnd_df.reset_index()

- Ghép tất cả vào ticker_dict

In [ ]:
weekly_other_ticker_dict = {}
number_of_days = 20

# Các index từ full_index_df
# Các ticker từ full_index_df
for ticker in ['VNINDEX', 'VN30', 'HNXINDEX', 'UPINDEX', 'VN30F1M', 'VN30F2M']:
    temp_df = full_index_df[full_index_df['ticker'] == ticker]
    temp_df = temp_df[['date','ticker','close']]
    temp_df = temp_df.sort_values('date', ascending=False).iloc[:number_of_days]
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    temp_df['1w_change'] = temp_df['close'][::-1].pct_change(4)[::-1].fillna(0)
    temp_df['1m_change'] = temp_df['close'][::-1].pct_change(19)[::-1].fillna(0)
    if ticker in ['VNINDEX', 'VN30']:
        temp_df['market'] = 'hose'
    elif ticker in ['HNXINDEX', 'UPINDEX']:
        temp_df['market'] = 'hnx'
    elif ticker in ['VN30F1M', 'VN30F2M']:
        temp_df['market'] = 'derivatives'
    temp_df['type'] = 'vn'
    weekly_other_ticker_dict[ticker] = temp_df.reset_index(drop=True)


# Các ticker từ other_ticker_df
for ticker in ['XAU_USD', 'CLZ', 'BTC_USD', 'ETH_USD', 'DJI', 'SPX', 'FTSE', 'STOXX50E', 'N225', 'SSEC']:
    temp_df = other_ticker_df[other_ticker_df['ticker'] == ticker]
    temp_df = temp_df[['date','ticker','close']]
    temp_df = temp_df.sort_values('date', ascending=False).iloc[:number_of_days]
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    temp_df['1w_change'] = temp_df['close'][::-1].pct_change(4)[::-1].fillna(0)
    temp_df['1m_change'] = temp_df['close'][::-1].pct_change(19)[::-1].fillna(0)
    if ticker in ['XAU_USD', 'CLZ']:
        temp_df['market'] = 'commodity'
        temp_df['type'] = 'other'
    elif ticker in ['BTC_USD', 'ETH_USD']:
        temp_df['market'] = 'crypto'
        temp_df['type'] = 'other'
    elif ticker in ['DJI', 'SPX']:
        temp_df['market'] = 'us'
        temp_df['type'] = 'international'
    elif ticker in ['FTSE', 'STOXX50E']:
        temp_df['market'] = 'eu'
        temp_df['type'] = 'international'
    elif ticker in ['N225', 'SSEC']:
        temp_df['market'] = 'asia'
        temp_df['type'] = 'international'
    weekly_other_ticker_dict[ticker] = temp_df.reset_index(drop=True)


#Thêm dữ liệu DXY và USD_VND
for ticker in ['DXY', 'USD_VND']:
    if ticker == 'DXY':
        temp_df = dxy_calculation_df.sort_values('date', ascending=False).iloc[:number_of_days]
    elif ticker == 'USD_VND':
        temp_df = usd_vnd_df.sort_values('date', ascending=False).iloc[:number_of_days]
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    temp_df['1w_change'] = temp_df['close'][::-1].pct_change(4)[::-1].fillna(0)
    temp_df['1m_change'] = temp_df['close'][::-1].pct_change(19)[::-1].fillna(0)
    temp_df['market'] = 'fx'
    temp_df['type'] = 'other'
    weekly_other_ticker_dict[ticker] = temp_df.reset_index(drop=True)

# Tạo bảng để lưu vào DB
weekly_history_data_df = pd.DataFrame()
for ticker, df in weekly_other_ticker_dict.items():
    weekly_history_data_df = pd.concat([weekly_history_data_df, df], ignore_index=True)

#### Phần nhận xét các Index

In [18]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))
model_list = get_gemini_models()
standard_model_list = select_standard_models(model_list)
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

def create_prompt(type_name):
    if type_name == 'vn':
        senerio = """
            Câu 1 (Thị trường chung): Bắt đầu bằng nhận định về chỉ số chính VN-Index, trích dẫn số liệu tăng trưởng tuần (1w_change).
            Câu 2 (Nhóm Cổ phiếu Lớn): Nhận xét về nhóm cổ phiếu blue-chip qua chỉ số VN30, trích dẫn số liệu tăng trưởng tuần (1w_change), nhấn mạnh mức độ biến động và so sánh với VN-Index.
            Câu 3 (Các Chỉ Số Khác): Mô tả xu hướng chung của hai chỉ số HNXINDEX và UPINDEX, trích dẫn số liệu 1w_change của chúng.
            Câu 4 (Thị trường Phái sinh): Nhận xét về thị trường phái sinh, chỉ nhận xét VN30F1M bỏ qua VN30F2M. So sánh điểm số đóng cửa của nó với chỉ số VN30 cơ sở để nêu bật mức chênh lệch (basis).
            Câu 5 (Kết luận): Viết một câu kết luận khách quan để tổng hợp lại bức tranh chung, ví dụ như mức độ lan tỏa của đà tăng hoặc vai trò dẫn dắt của nhóm cổ phiếu nào.
        """
    elif type_name == 'international':
        senerio = """
            Câu 1: Nhận định chung về thị trường chứng khoán toàn cầu trong tuần qua.
            Câu 2: Tập trung vào thị trường Mỹ (DJI, SPX) và trích dẫn số liệu 1w_change.
            Câu 3: Tập trung vào thị trường Châu Âu (FTSE, STOXX50E) và trích dẫn số liệu 1w_change.
            Câu 4: Tập trung vào thị trường Châu Á (N225, SSEC) và trích dẫn số liệu 1w_change.
            Câu 5: Viết một câu kết luận cuối cùng. Câu này phải **hoàn toàn khách quan**, chỉ tổng hợp lại thông tin, **tuyệt đối không đưa ra quan điểm cá nhân hay dự đoán**.
        """
    elif type_name == 'other':
        senerio = """
            Câu 1: Nhận định chung về các loại thị trường trong tuần qua (Crypto, Hàng hóa, Ngoại hối).
            Câu 2: Tập trung vào thị trường Tiền điện tử (BTC, ETH) và trích dẫn số liệu 1w_change.
            Câu 3: Tập trung vào thị trường Hàng hóa (Vàng, Dầu) và trích dẫn số liệu 1w_change.
            Câu 4: Tập trung vào thị trường Ngoại hối (DXY, USD/VND) và trích dẫn số liệu 1w_change.
            Câu 5: Viết một câu kết luận cuối cùng. Câu này phải **hoàn toàn khách quan**, chỉ tổng hợp lại thông tin, **tuyệt đối không đưa ra quan điểm cá nhân hay dự đoán**.
        """

    #Tách bảng lớn ra thành nhiều bảng để AI dễ đọc
    type_df = weekly_history_data_df[weekly_history_data_df['type'] == type_name]
    type_dict = {}
    index_list = type_df['ticker'].unique().tolist()
    for index in index_list:
        temp_df = type_df[type_df['ticker'] == index]
        temp_df = temp_df.sort_values('date', ascending=False).reset_index(drop=True)
        type_dict[index] = temp_df.to_csv(index=False, sep='|', lineterminator='\n')

    prompt = f"""
        Dựa vào các bảng dữ liệu được cung cấp dưới đây, hãy viết một đoạn văn nhận xét thị trường:
        Index số 1: {type_dict[index_list[0]]}\n
        Index số 2: {type_dict[index_list[1]]}\n
        Index số 3: {type_dict[index_list[2]]}\n
        Index số 4: {type_dict[index_list[3]]}\n
        Index số 5: {type_dict[index_list[4]]}\n
        Index số 6: {type_dict[index_list[5]]}\n
        ---
        **Yêu cầu BẮT BUỘC:**

        1.  **Định dạng:** Toàn bộ nội dung phải nằm trong **một đoạn văn duy nhất**.
        2.  **Độ dài:** Mỗi câu có độ dài khoảng **15 đến 20 từ**.
        3.  **Cấu trúc & Nội dung:** Đoạn văn phải có **chính xác 5 câu** theo kịch bản dưới đây.
            {senerio}
        4.  Văn phong & Sáng tạo:
            Mục tiêu: Hành văn phải chuyên nghiệp, trôi chảy và khách quan như một nhà phân tích thị trường, với các nhận định phản ánh đúng bối cảnh của dữ liệu.
            Quy tắc 1: Phân tích Bối cảnh (Rất quan trọng):
                - Việc lựa chọn từ ngữ mô tả xu hướng (ví dụ: 'phục hồi', 'tăng trưởng') phải dựa trên diễn biến giá trong lịch sử gần đây có trong dữ liệu, không chỉ dựa vào một con số của tuần gần nhất.
                - Ví dụ cụ thể: Chỉ được dùng từ "phục hồi" hoặc "hồi phục" khi thị trường vừa trải qua một đợt sụt giảm rõ rệt trước đó. Nếu thị trường vốn đang đi lên và tiếp tục tăng mạnh, phải dùng các từ như "tăng trưởng mạnh", "bứt phá", hoặc "nới rộng đà tăng".
            Quy tắc 2: Thuật ngữ Chuẩn xác:
                - Ưu tiên sử dụng các thuật ngữ tài chính khách quan, định lượng. Tránh các từ ngữ cảm tính hoặc không phù hợp (ví dụ: không dùng "sôi động", "tiến bộ").
            Quy tắc 3: Chống lặp từ:
                - Tuyệt đối không lặp lại từ ngữ, cụm từ và cấu trúc câu trong toàn bộ đoạn văn để đảm bảo sự linh hoạt và tự nhiên.
            Quy tắc 4: Từ ngữ không được sử dụng:
                - Tuyệt đối không sử dụng từ "Tóm lại" trong bất kỳ trường hợp
            Quy tắc 5: Trích xuất số liệu:
                - Các số liệu gốc ở dạng só thập phân khi trích dẫn phải ghi về dạng phần trăm, ví dụ: 0.05 phải ghi là 5%, 0.1234 phải ghi là 12.34%.
                - Không ghi số liệu dạng chữ, ví dụ: "tăng 5%" thay vì "tăng năm phần trăm".
    """
    return prompt

vn_comment = generate_content_with_model_dict(standard_model_dict, create_prompt('vn'))
international_comment = generate_content_with_model_dict(standard_model_dict, create_prompt('international'))
other_comment = generate_content_with_model_dict(standard_model_dict, create_prompt('other'))

### Phần phân tích thị trường

#### Biến động các chỉ số CKVN

In [20]:
# Dùng chung bảng weekly_history_data_df

#### Khối ngoại và tự doanh

- Lịch sử mua bán ròng trong tuần

In [21]:
weekly_nntd_history_df = nntd_index_df[nntd_index_df['ticker'].isin(['VNINDEX','HNXINDEX', 'UPINDEX'])][['date', 'type', 'buy_value', 'sell_value', 'net_value']]
weekly_nntd_history_df = weekly_nntd_history_df.groupby(['date', 'type']).sum().sort_values('date', ascending=False).reset_index()
latest_dates = weekly_nntd_history_df['date'].drop_duplicates().sort_values(ascending=False).head(5)
weekly_nntd_history_df = weekly_nntd_history_df[weekly_nntd_history_df['date'].isin(latest_dates)]

- Top cổ phiếu mua bán ròng

In [22]:
temp_df_list = []

for stock_type in ['NN', 'TD']:
    type_df = pd.DataFrame()
    
    for ticker in nntd_stock_df['ticker'].unique():
        temp_df = nntd_stock_df[(nntd_stock_df['ticker'] == ticker) & (nntd_stock_df['type'] == stock_type)]
        temp_df = temp_df.sort_values('date', ascending=False).reset_index(drop=True)
        
        if len(temp_df) == 0:
            continue
            
        temp_df['weekly_value'] = temp_df['net_value'][::-1].rolling(window=5, min_periods=1).sum()[::-1]
        type_df = pd.concat([type_df, temp_df[['type', 'ticker', 'weekly_value']].iloc[[0]]], ignore_index=True)
    
    type_df = type_df.sort_values('weekly_value', ascending=False).reset_index(drop=True)
    temp_df_list.append(pd.concat([type_df.head(10), type_df.tail(10)], ignore_index=True))

# Gán kết quả và tổng hợp
weekly_nntd_stock_df = pd.concat(temp_df_list, ignore_index=True)

#### Biến động vốn hoá

In [23]:
# --- PHẦN 1: TÍNH TOÁN NET_CAP CHO TỪNG CỔ PHIẾU ---
open_week_date = full_stock_df['date'].min()
close_week_date = full_stock_df['date'].max()

open_week_stock_cap_df = full_stock_df[full_stock_df['date'] == open_week_date][['ticker', 'cap']].rename(columns={'cap': 'open_cap'})
close_week_stock_cap_df = full_stock_df[full_stock_df['date'] == close_week_date][['ticker', 'cap']].rename(columns={'cap': 'close_cap'})

stock_cap_df = pd.merge(open_week_stock_cap_df, close_week_stock_cap_df, on='ticker', how='outer')
stock_cap_df.fillna({'open_cap': 0, 'close_cap': 0}, inplace=True)
stock_cap_df['net_cap'] = stock_cap_df['close_cap'] - stock_cap_df['open_cap']
stock_cap_df['industry'] = stock_cap_df['ticker'].map(full_stock_classification_df.set_index('ticker')['industry_name']).map(name_map_dict)
stock_cap_df = stock_cap_df.sort_values('net_cap', ascending=False).reset_index(drop=True)

# --- PHẦN 2: TÍNH TOÁN TỔNG LỰC KÉO/ĐẨY TOÀN THỊ TRƯỜNG ---
# Vẫn tính toán các giá trị này như biến thông thường để sử dụng
gainers_df = stock_cap_df[stock_cap_df['net_cap'] > 0]
losers_df = stock_cap_df[stock_cap_df['net_cap'] < 0]

cap_gain = gainers_df['net_cap'].sum()
cap_loss = abs(losers_df['net_cap'].sum()) # Dùng abs() để mẫu số luôn dương

# --- PHẦN 3: TỔNG HỢP DỮ LIỆU THỊ TRƯỜNG, NGÀNH, CỔ PHIẾU ---
# 3.1. Tổng hợp cho toàn thị trường
market_cap_sum = stock_cap_df.select_dtypes(include=[np.number]).sum(axis=0)
change_market_cap_df = pd.DataFrame([market_cap_sum])
change_market_cap_df['ticker'] = 'Thị trường'
change_market_cap_df['type'] = 'market'

# 3.2. Tổng hợp cho các ngành
change_industry_cap_df = (
    stock_cap_df
    .drop(columns=['ticker'])
    .groupby('industry').sum().reset_index()
    .rename(columns={'industry': 'ticker'})
    .sort_values('net_cap', ascending=False)
)
change_industry_cap_df['type'] = 'industry'

# 3.3. Tổng hợp cho top cổ phiếu
change_stock_cap_df = pd.concat([
    stock_cap_df.head(5).drop(columns=['industry']),
    stock_cap_df.tail(5).drop(columns=['industry'])],
    ignore_index=True
)
change_stock_cap_df['type'] = 'stock'

# 3.4. Ghép các bảng lại
market_cap_change_df = pd.concat([change_market_cap_df, change_industry_cap_df, change_stock_cap_df], ignore_index=True)

# --- PHẦN 4: TÍNH TOÁN CÁC CHỈ SỐ CUỐI CÙNG --- ## MODIFIED ##
# ## NEW ##: Thêm 2 cột mới để chứa giá trị tổng lực kéo và đẩy
# Các giá trị này sẽ được lặp lại trên tất cả các hàng
market_cap_change_df['cap_gain'] = cap_gain
market_cap_change_df['cap_loss'] = cap_loss

# Tính % thay đổi vốn hóa (giữ nguyên)
market_cap_change_df['change_cap'] = np.divide(
    (market_cap_change_df['close_cap'] - market_cap_change_df['open_cap']),
    market_cap_change_df['open_cap']
)
market_cap_change_df['change_cap'].replace([np.inf, -np.inf], 0, inplace=True)

# Tính toán tác động Kéo/Đẩy
market_cap_change_df['impact_group'] = 'Không đổi'
market_cap_change_df['impact_ratio'] = 0.0

# Áp dụng logic cho nhóm "positive"
if cap_gain > 0:
    gainers_mask = market_cap_change_df['net_cap'] > 0
    market_cap_change_df.loc[gainers_mask, 'impact_group'] = 'positive'
    # ## MODIFIED ##: Chia cho cột 'cap_gain' thay vì biến
    market_cap_change_df.loc[gainers_mask, 'impact_ratio'] = market_cap_change_df['net_cap'] / market_cap_change_df['cap_gain']

# Áp dụng logic cho nhóm "negative"
if cap_loss > 0:
    losers_mask = market_cap_change_df['net_cap'] < 0
    market_cap_change_df.loc[losers_mask, 'impact_group'] = 'nagative' # "nagative" -> "negative"
    # ## MODIFIED ##: Chia cho cột 'cap_loss' thay vì biến
    market_cap_change_df.loc[losers_mask, 'impact_ratio'] = market_cap_change_df['net_cap'] / market_cap_change_df['cap_loss']

# Xử lý riêng cho dòng 'Thị trường'
market_mask = market_cap_change_df['type'] == 'market'
market_cap_change_df.loc[market_mask, 'impact_group'] = 'neutral'
market_cap_change_df.loc[market_mask, 'impact_ratio'] = 1.0

# ## MODIFIED ##: Sắp xếp lại các cột cuối cùng, thêm 2 cột mới vào
market_cap_change_df = market_cap_change_df[[
    'type', 'ticker', 'open_cap', 'close_cap', 
    'cap_gain', 'cap_loss', 'net_cap', 'change_cap',
    'impact_group', 'impact_ratio'
]]

#### Dữ liệu VNINDEX và MS

In [24]:
ta_vn_index_df = full_index_df[full_index_df['ticker'] == 'VNINDEX'].reset_index(drop=True).iloc[:80]
ms_vn_index_df = full_ms_chart_df.iloc[:80]
weekly_vn_index_df = pd.merge(ta_vn_index_df, ms_vn_index_df, on='date', how='left')

#### Nhận xét VNINDEX và MS

In [25]:
prompt = f"""
    Đây là dữ liệu của chỉ số VNINDEX:
        {ta_vn_index_df.to_csv(index=False, sep='|', lineterminator='\n')}
    **Vai trò:** Bạn là một **nhà phân tích kỹ thuật cấp cao**, có kinh nghiệm tại một công ty chứng khoán.
    **Nhiệm vụ:** Dựa trên dữ liệu biến động giá và các chỉ báo kỹ thuật của chỉ số VNINDEX được cung cấp, hãy viết một báo cáo phân tích kỹ thuật chuyên nghiệp, súc tích và liền mạch, tuân thủ nghiêm ngặt cấu trúc 8 câu dưới đây.
    **Yêu cầu về nội dung, cấu trúc và định dạng:**
        Báo cáo phải gồm **8 câu, mỗi câu dài khoảng 20 từ** và được chia thành **3 đoạn văn riêng biệt**. Bắt đầu thẳng vào nội dung phân tích, không có lời chào.
        **Đoạn 1 (3 câu):**
            * **Câu 1:** Nhận định tổng quan về xu hướng chính của chỉ số trong **tuần vừa qua**.
            * **Câu 2:** Phân tích vai trò của hai đường **SMA 20 và SMA 60** (hỗ trợ/kháng cự).
            * **Câu 3:** Đánh giá chỉ báo **RSI 14** và ý nghĩa của nó đối với áp lực thị trường.
        **Đoạn 2 (3 câu):**
            * **Câu 4:** Phân tích mốc hỗ trợ/kháng cự quan trọng theo **khung Tuần** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
            * **Câu 5:** Phân tích mốc hỗ trợ/kháng cự quan trọng theo **khung Tháng** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
            * **Câu 6:** Phân tích mốc hỗ trợ/kháng cự quan trọng theo **khung Quý** (kết hợp O-H-L gần nhất và Fibonacci gần nhất).
        **Đoạn 3 (2 câu):**
            * **Câu 7:** Tổng hợp và nhấn mạnh **một mốc hỗ trợ hoặc kháng cự cốt lõi nhất** cần theo dõi.
            * **Câu 8:** Đề xuất **chiến lược giao dịch** cho tuần tới một cách rõ ràng, dứt khoát.
        **Yêu cầu về ngôn ngữ và trình bày:**
            * **Bắt buộc trích dẫn số liệu cụ thể và đa dạng:**
                * **Với mỗi ngưỡng hỗ trợ/kháng cự, hãy linh hoạt trích dẫn khoảng cách tới giá đóng cửa. Sử dụng xen kẽ giữa độ lệch phần trăm (%) và độ lệch điểm tuyệt đối để tránh nhàm chán trong văn phong.** Hãy diễn đạt một cách tự nhiên, ví dụ: "...tại 1381.12, thấp hơn 5.2% so với giá hiện tại" hoặc "...quanh 1392 điểm, cách giá đóng cửa hơn 65 điểm".
                * Các nhận xét đưa ra đều phải kèm theo số liệu của chỉ báo.
            * **Yêu cầu về văn phong và diễn đạt:**
                * Sử dụng văn phong **sắc bén, quyết đoán và có chiều sâu**.
                * **Sử dụng cấu trúc câu và từ vựng đa dạng.** Tránh lặp lại một mẫu câu hoặc một từ nhiều lần.
                * **Liên kết các câu văn một cách mượt mà** bằng các từ/cụm từ chuyển tiếp để tạo thành một dòng chảy phân tích liền mạch.
            * **Yêu cầu về định dạng và thuật ngữ:**
                * **TUYỆT ĐỐI KHÔNG được viết tên cột dữ liệu gốc (ví dụ: `WFIBO_0382`, `month_prev_high`). BẮT BUỘC phải diễn giải chúng sang ngôn ngữ phân tích chuyên nghiệp (ví dụ: 'ngưỡng Fibonacci 38.2% của khung tuần', 'đỉnh giá của tháng trước').**
                * **TUYỆT ĐỐI KHÔNG** bao gồm lời mở đầu như "Kính gửi Quý Khách hàng" hoặc "Dựa trên dữ liệu được cung cấp:", hãy bắt đầu ngay vào nội dung phân tích.
"""
ta_vnindex_comment = generate_content_with_model_dict(standard_model_dict, prompt)

In [26]:
prompt = f"""
### **Phần 1: Bối cảnh và Diễn giải Dữ liệu (Kiến thức nền cho AI)**

Trước khi thực hiện nhiệm vụ, hãy nghiên cứu và hiểu rõ bản chất của bộ dữ liệu dưới đây.

#### **I. Tổng quan**
Bộ dữ liệu này thể hiện các **đường xu hướng** thị trường qua các khung thời gian khác nhau. Mục tiêu là cung cấp một cái nhìn đa chiều, giúp xác định sức mạnh, sự bền vững của xu hướng và các vùng cần chú ý đặc biệt.

#### **II. Cấu trúc Dữ liệu**
* `date`: Ngày ghi nhận dữ liệu.
* `trend_5p`: **Đường xu hướng** cho khung thời gian **tuần**.
* `trend_20p`: **Đường xu hướng** cho khung thời gian **tháng**.
* `trend_60p`: **Đường xu hướng** cho khung thời gian **quý**.

#### **III. Diễn giải Giá trị của các Đường xu hướng**
Tất cả các cột `trend_*` là giá trị thập phân từ 0 đến 1, **tương ứng với 0% đến 100%**. **Khi viết bài phân tích, bạn phải luôn chuyển đổi và sử dụng định dạng phần trăm (ví dụ: 0.62 sẽ được viết là 62%).**

* **Tiến về 100%**: Thể hiện một **xu hướng tăng** đang mạnh dần lên.
* **Tiến về 0%**: Thể hiện một **xu hướng giảm** đang mạnh dần lên.
* **Ngưỡng `80%`**: Một **vùng giá trị cao**, cho thấy đà tăng đã rất mạnh. Việc mua vào tại vùng này có thể gặp bất lợi về giá (mua đuổi) và cần sự thận trọng.
* **Ngưỡng `20%`**: Một vùng giá trị thấp và được xem là **vùng rủi ro cao**. Bán ra tại đây có rủi ro bán đúng đáy, trong khi việc mua vào cũng cần hết sức thận trọng vì xu hướng giảm có thể chưa kết thúc.

#### **IV. Các Nguyên tắc Phân tích Chính**
1.  **Nguyên tắc Chu kỳ:** Các đường xu hướng có xu hướng di chuyển theo chu kỳ giữa vùng giá trị thấp và cao.
2.  **Nguyên tắc Đồng pha:** Xu hướng bền vững khi có sự đồng thuận từ nhiều đường xu hướng.
3.  **Nguyên tắc Chi phối:** Khung thời gian lớn hơn sẽ chi phối khung nhỏ hơn.
4.  **Nguyên tắc Hỗ trợ/Kháng cự:** Các ngưỡng **20%** và **80%** là các mốc tâm lý quan trọng.

---

### **Phần 2: Yêu cầu Phân tích (Nhiệm vụ cần thực hiện)**

**Dữ liệu:**
{ms_vn_index_df.drop(columns=['trend_240p']).to_csv(index=False, sep='|', lineterminator='\n')}

**Vai trò:** Bạn là một **nhà phân tích chiến lược thị trường cấp cao**, với văn phong **dứt khoát, súc tích và sắc bén.**

**Nhiệm vụ:** Dựa trên kiến thức về bộ dữ liệu và bảng dữ liệu được cung cấp, hãy viết một báo cáo phân tích xu hướng thị trường ngắn gọn.

**Yêu cầu về nội dung và cấu trúc:**

Báo cáo cần được chia thành **2 đoạn văn ngắn gọn, đi thẳng vào vấn đề.** Bắt đầu ngay lập tức, không có lời chào.

* **Đoạn 1: Hiện trạng và Diễn giải.**
    * Nêu sự thay đổi nổi bật nhất của **xu hướng tuần** và đặt nó trong sự tương phản trực tiếp với các xu hướng dài hơn.
    * Dựa trên nguyên tắc chi phối và sự phân kỳ, **đưa ra kết luận ngay lập tức** rằng đây là một pha điều chỉnh kỹ thuật hay một tín hiệu cảnh báo rủi ro.

* **Đoạn 2: Hàm ý và Chiến lược Hành động.**
    * Từ kết luận trên, nêu **hàm ý cốt lõi** đối với thị trường (ví dụ: rủi ro ngắn hạn tăng nhưng xu hướng lớn chưa thay đổi).
    * Đề xuất một **chiến lược hành động duy nhất, rõ ràng và dứt khoát** cho tuần tới.

**Yêu cầu về ngôn ngữ và trình bày:**

* **Văn phong:** **Dứt khoát, súc tích và đi thẳng vào vấn đề. Mỗi câu văn phải mang một thông tin cốt lõi, tránh các diễn giải rườm rà, không cần thiết.**
* **Sử dụng thuật ngữ đa dạng:** Hạn chế lặp lại nguyên văn tên các đường xu hướng.
* **Trích dẫn số liệu:** Các nhận xét phải kèm theo số liệu dưới **định dạng phần trăm**.
* **Định dạng và Thuật ngữ:**
    * **TUYỆT ĐỐI KHÔNG** được viết tên cột dữ liệu gốc (ví dụ: `trend_5p`).
    * **TUYỆT ĐỐI KHÔNG SỬ DỤNG TỪ 'CHỈ BÁO' (INDICATOR).** Đây là các **'đường xu hướng' (trend lines)**. Hãy luôn sử dụng đúng thuật ngữ này.
    * **TUYỆT ĐỐI KHÔNG** bao gồm lời mở đầu hoặc kết thúc.
    """

ms_chart_comment = generate_content_with_model_dict(standard_model_dict, prompt)

### Phần danh mục khuyến nghị

In [28]:
weekly_portfolio_df = pd.read_excel('../data/weekly/cts_portfolio.xlsx')
weekly_portfolio_df['industry'] = weekly_portfolio_df['ticker'].map(full_stock_classification_df.set_index('ticker')['industry_name']).map(name_map_dict)
weekly_portfolio_df = weekly_portfolio_df.sort_values('industry')

t5_pct_change_list = []
for ticker in weekly_portfolio_df['ticker'].unique():
    temp_df = full_stock_df[full_stock_df['ticker'] == ticker]
    temp_df['t5_pct_change'] = temp_df['close'][::-1].pct_change(4).fillna(0)[::-1]
    t5_pct_change_list.append(temp_df['t5_pct_change'].iloc[0])

weekly_portfolio_df['t5_pct_change'] = t5_pct_change_list
weekly_portfolio_df = weekly_portfolio_df.merge(today_stock_df[['ticker', 't5_score']], on='ticker', how='left')

### Lưu vào MSSQL


In [32]:
weekly_comments_df = pd.DataFrame({
    'type': ['vn', 'international', 'other', 'ta_vnindex', 'ms_chart'],
    'group': ['news', 'news', 'news', 'chart', 'chart'],
    'comment': [vn_comment, international_comment, other_comment, ta_vnindex_comment, ms_chart_comment],
})

In [37]:
%%capture
save_to_mssql(cts_engine, weekly_history_data_df, 'weekly_history_data')
save_to_mssql(cts_engine, weekly_nntd_history_df, 'weekly_nntd_history')
save_to_mssql(cts_engine, weekly_nntd_stock_df, 'weekly_nntd_stock')
save_to_mssql(cts_engine, weekly_vn_index_df, 'weekly_vn_index')
save_to_mssql(cts_engine, weekly_comments_df, 'weekly_comments')